# ZAlign

In [4]:
from zalign import ZScore
from rdkit import Chem
import json
import os
import numpy as np
def show_similarity_matrix(matrix):
    # 用plotly绘制相似性得分矩阵的热力图，可以交互式查看横纵坐标
    import plotly.graph_objs as go
    import numpy as np

    # 构造x轴和y轴标签（原子索引从1开始）
    x_labels = [str(i+1) for i in range(matrix.shape[1])]
    y_labels = [str(i+1) for i in range(matrix.shape[0])]

    heatmap = go.Heatmap(
        z=matrix,
        x=x_labels,
        y=y_labels,
        colorscale='Hot',
        colorbar=dict(title='Score'),
        hoverongaps=False,
        hovertemplate='Query Atom: %{y}<br>Template Atom: %{x}<br>Score: %{z:.1f}<extra></extra>'
    )

    layout = go.Layout(
        title='Similarity Score Matrix',
        xaxis=dict(title='Template Molecule Atom Index'),
        yaxis=dict(title='Query Molecule Atom Index'),
        width=900,
        height=900
    )

    fig = go.Figure(data=[heatmap], layout=layout)
    fig.show()



def generate_zalign_input(fp_sdf_q, fp_sdf_t, fp_json_ori, fp_json_align):
    """ Args:
        fp_sdf_q: str, path to query sdf file
        fp_sdf_t: str, path to template sdf file
        fp_json_ori: str, path to original json file (unidock2-dumped input json for ud2 engine)
        fp_json_align: str, path to align json file (input json for ud2 engine, with bias info, for alignment)
    
    TODO: multiple template molecules

    """
    list_mol_q = Chem.SDMolSupplier(fp_sdf_q, removeHs=False)
    mol_t = Chem.SDMolSupplier(fp_sdf_t, removeHs=False)[0]

    # TODO: The order of molecule name should be CHECKED
    #------- 准备输入JSON
    with open(fp_json_ori, "r") as f:
        data = json.load(f)
    list_k = [k for k in data.keys() if k not in {"score", "receptor"}]
    list_matrix = []

    for mol_q, k in zip(list_mol_q, list_k):
        matrix = ZScore(mol_q, mol_t).score()
        list_matrix.append(matrix)

        # 得到所有原子的坐标
        coords_t = mol_t.GetConformer().GetPositions().tolist()
        coords_q = mol_q.GetConformer().GetPositions().tolist()
        center_t = np.mean(coords_t, axis=0)
        center_q = np.mean(coords_q, axis=0)
        translation = center_t - center_q
        
        natom_t = mol_t.GetNumAtoms()
        natom_q = mol_q.GetNumAtoms()

        for i in range(natom_q):

            biases = []
            for j in range(natom_t): # each template atom
                biases.append([*coords_t[j], float(matrix[i][j]), 0.])
            
            # add biases to this atom, supposing no bias info by default
            data[k]["atoms"][i].append(biases)

            # Align two centers
            data[k]["atoms"][i][0] += translation[0]
            data[k]["atoms"][i][1] += translation[1]
            data[k]["atoms"][i][2] += translation[2]

        data["score"] = ["vina", "gaff2"] # `unidock2`-dumped json losts this key 
        with open(fp_json_align, "w") as f:
            json.dump(data, f)
    
    return list_mol_q, mol_t, list_matrix

### 分体流程

In [35]:
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_4IEH_ref/"
fp_sdf_q = os.path.join(dp, "CHEMBL376408_prepared.sdf")
fp_sdf_t = os.path.join(dp, "1E9_ref.sdf")
dp_tmp = os.path.join(dp, "tmp/")
fp_json_ori = os.path.join(dp_tmp, sorted(os.listdir(dp_tmp))[-1], "ud2_engine_inputs.json")
fp_json_align = os.path.join(dp, "ud2_align.json")

mol_q, mol_t, matrix = generate_zalign_input(fp_sdf_q, fp_sdf_t, fp_json_ori, fp_json_align)

In [ ]:
show_similarity_matrix(matrix)

## 自动执行 align docking 的流程

In [1]:
import subprocess as sp
import yaml
import os
import time
import shutil

# ------------- Configurations
fp_ud2 = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/cmake-build-release/bin/ud2"
fp_json2sdf = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/scripts/json2sdf.py"
dp_config = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align/align_template"

def get_config(type:str):
    if type == "free":
        with open(os.path.join(dp_config, "unidock2_free.yaml"), "r") as f: 
            config_free = yaml.safe_load(f)
        return config_free
    
    elif type == "mcs": 
        with open(os.path.join(dp_config, "unidock2_mcs.yaml"), "r") as f:
            config_mcs = yaml.safe_load(f)
        return config_mcs

    elif type == "ud2":
        with open(os.path.join(dp_config, "ud2.yaml"), "r") as f:
            config_ud2 = yaml.safe_load(f)
        return config_ud2
    
    else:
        raise ValueError(f"Invalid config type: {type}")


def generate_rand(dp:str, fn_pdb:str, fn_sdf_q:str, center:list):

    time_start = time.time()

    # ------------- 执行 unidock2 MCS docking
    fp_config_unidock2 = os.path.join(dp, "unidock2_rand.yaml")
    
    config_rand = get_config("free")
    config_rand["Advanced"]["exhaustiveness"] = 10
    config_rand["Advanced"]["mc_steps"] = 1
    config_rand["Advanced"]["num_pose"] = 1
    config_rand["Advanced"]["opt_steps"] = 1

    fn_sdf_q_rand = fn_sdf_q[:-4] + "_rand.sdf"
    config_rand["Preprocessing"]["output_docking_pose_sdf_file_name"] = fn_sdf_q_rand
    config_rand["Settings"]["task"] = "screen"

    config_rand["Required"]["receptor"] = fn_pdb
    config_rand["Required"]["ligand"] = fn_sdf_q
    config_rand["Required"]["center"] = center
    
    with open(fp_config_unidock2, "w") as f:
        yaml.dump(config_rand, f)

    sp.run(f"unidock2 docking -cf {fp_config_unidock2}", shell=True, cwd=dp)


    time_end = time.time()
    print(f"\n\nGenerate random pose time: {time_end - time_start} seconds\n\n")
    return fn_sdf_q_rand


def perform_align_docking(dp:str, fn_pdb:str, fn_sdf_q:str, fn_sdf_t:str, center:list, 
    exhaustiveness:int=512, bias_k:float=0.3, topN:int=5):

    import traceback

    time_start = time.time()

    # 打开 align.log 文件用于记录所有命令行输出
    fp_log = os.path.join(dp, "align.log")
    if os.path.exists(fp_log):
        os.remove(fp_log)
    fp_ud2_log = os.path.join(dp, "ud.log")
    if os.path.exists(fp_ud2_log):
        os.remove(fp_ud2_log)

    log_file = open(fp_log, "a")
    try:
        # ------------- 执行 unidock2 MCS docking
        fp_config_unidock2 = os.path.join(dp, "unidock2_mcs.yaml")
        config_mcs = get_config("mcs")
        config_mcs["Required"]["receptor"] = fn_pdb
        config_mcs["Required"]["ligand"] = fn_sdf_q
        config_mcs["Required"]["center"] = center
        config_mcs["Preprocessing"]["reference_sdf_file_name"] = fn_sdf_t
        with open(fp_config_unidock2, "w") as f:
            yaml.dump(config_mcs, f)

        sp.run(f"unidock2 docking -cf {fp_config_unidock2}", shell=True, cwd=dp, stdout=log_file, stderr=log_file)
        
        time_end = time.time()
        print(f"\n\nMCS docking cost: {time_end - time_start} seconds\n\n")
        log_file.write(f"\n\nMCS docking cost: {time_end - time_start} seconds\n\n")
        time_start = time.time()

        # ------------- 执行 unidock2, 生产 JSON 输入文件
        dp_tmp = os.path.join(dp, "tmp/")
        if os.path.exists(dp_tmp):
            shutil.rmtree(dp_tmp)
        os.makedirs(dp_tmp, exist_ok=True)
        fp_config_unidock2 = os.path.join(dp, "unidock2_free.yaml")
        config_free = get_config("free")
        config_free["Preprocessing"]["temp_dir_name"] = "./tmp"
        config_free["Required"]["receptor"] = fn_pdb
        config_free["Required"]["ligand"] = fn_sdf_q
        config_free["Required"]["center"] = center
        with open(fp_config_unidock2, "w") as f:
            yaml.dump(config_free, f)

        sp.run(f"unidock2 docking -cf {fp_config_unidock2}", shell=True, cwd=dp, stdout=log_file, stderr=log_file)

        # ------------- 加工输入 JSON，添加bias
        fp_sdf_q = os.path.join(dp, fn_sdf_q)
        fp_sdf_t = os.path.join(dp, fn_sdf_t)
        fp_json_ori = os.path.join(dp_tmp, sorted(os.listdir(dp_tmp))[-1], "ud2_engine_inputs.json")
        fp_json_align = os.path.join(dp, "ud2_align.json")

        list_mol_q, mol_t, list_matrix = generate_zalign_input(fp_sdf_q, fp_sdf_t, fp_json_ori, fp_json_align)

        # ------------- 执行 ud2, 生产json输出文件
        fp_config_ud2 = os.path.join(dp, "ud2.yaml")
        config_ud2 = get_config("ud2")
        config_ud2["Settings"]["center_x"] = center[0]
        config_ud2["Settings"]["center_y"] = center[1]
        config_ud2["Settings"]["center_z"] = center[2]
        config_ud2["Advanced"]["bias_k"] = bias_k
        config_ud2["Advanced"]["exhaustiveness"] = exhaustiveness
        config_ud2["Advanced"]["num_pose"] = topN
        with open(fp_config_ud2, "w") as f:
            yaml.dump(config_ud2, f)
        sp.run(f"{fp_ud2} {fp_config_ud2}", shell=True, cwd=dp, stdout=log_file, stderr=log_file)

        # ------------- 将输出 JSON 文件转换为 SDF 文件
        fp_json_align = os.path.join(dp, "ud2_align.json")

        dp_res_ud2 = os.path.join(dp, "res2")
        with open(fp_json_align, "r") as f:
            data = json.load(f)
        list_k = [k for k in data.keys() if k not in {"score", "receptor"}]
        data_res = dict()
        for fn_out in os.listdir(dp_res_ud2):
            fp_tmp = os.path.join(dp_res_ud2, fn_out)
            with open(fp_tmp, "r") as f:
                data_res.update(json.load(f))
                
        data_res = {k: data_res[k] for k in list_k}
        fp_res_ud2 = os.path.join(dp, "ud2_align_out.json")
        with open(fp_res_ud2, "w") as f:
            json.dump(data_res, f)

        sp.run(f"python {fp_json2sdf} {fp_res_ud2} {fp_sdf_q} ud2_out_bias.sdf", shell=True, cwd=dp, stdout=log_file, stderr=log_file)

        time_end = time.time()
        print(f"\n\nAlign docking cost: {time_end - time_start} seconds\n\n")
        log_file.write(f"\n\nAlign docking cost: {time_end - time_start} seconds\n\n")
    except Exception as e:
        # 打印异常到控制台
        print("ERRORS OCCURRED: ", str(e))
        traceback.print_exc()
        # 同时写入日志文件
        log_file.write("\n\nERRORS OCCURRED: {}\n".format(str(e)))
        log_file.write(traceback.format_exc())
        log_file.flush()
        raise
    finally:
        log_file.close()

    return list_mol_q, mol_t, list_matrix

### 4IEH

In [33]:
# ------------- User Input
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align/align_4IEH/diff_ring/"
fn_pdb = "4IEH.pdb"
fn_sdf_q = "CHEMBL376408_0.6_nM_3d_dock.sdf"
fn_sdf_t = "CHEMBL2322026_8.0_nM_3d_dock.sdf"
center = [12, 25, 11]

fn_sdf_q_rand = generate_rand(dp, fn_pdb, fn_sdf_q, center)
fn_sdf_q = fn_sdf_q_rand
mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 2048, 0.3)




    ██╗   ██╗██████╗ ██████╗
    ██║   ██║██╔══██╗╚════██╗
    ██║   ██║██║  ██║ █████╔╝
    ██║   ██║██║  ██║██╔═══╝
    ╚██████╔╝██████╔╝███████╗
     ╚═════╝ ╚═════╝ ╚══════╝

    DP Technology Docking Toolkit


{'receptor': '4IEH.pdb', 'ligand': 'CHEMBL376408_0.6_nM_3d_dock.sdf', 'ligand_batch': None, 'center': [12.0, 25.0, 11.0], 'exhaustiveness': 10, 'randomize': True, 'mc_steps': 1, 'opt_steps': 1, 'refine_steps': 0, 'num_pose': 1, 'rmsd_limit': 1.0, 'energy_range': 3.0, 'seed': 12345, 'use_tor_lib': False, 'n_cpu': None, 'gpu_device_id': 0, 'box_size': [30.0, 30.0, 30.0], 'task': 'screen', 'search_mode': 'free', 'template_docking': False, 'reference_sdf_file_name': None, 'compute_center': True, 'core_atom_mapping_dict_list': None, 'covalent_ligand': False, 'covalent_residue_atom_info_list': None, 'preserve_receptor_hydrogen': False, 'temp_dir_name': './tmp', 'output_receptor_dms_file_name': 'receptor_parameterized.dms', 'output_docking_pose_sdf_file_name': 'CHEMBL376408_0.6_n

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:442: UserWarning: PDB file contained CONECT record to TER entry. These are not included in bonds.
  warnings.warn(
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-09-02 13:46:25.153] [info] ----------------------- RUN Screening -----------------------
[2025-09-02 13:46:25.153] [info] Receptor loaded: 1100 atoms in box
[2025-09-02 13:46:25.154] [info] Ligands loaded. Total count: 1
[2025-09-02 13:46:25.210] [info] ----------------------- RUN Screening -----------------------
[2025-09-02 13:46:25.210] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=-3 Angstrom, x_hi=27 Angstrom, y_lo=10 Angstrom, y_hi=40 Angstrom, z_lo=-4 Angstrom, z_hi=26 Angstrom, 
constraint_docking=false, exhaustiveness=10, 
mc_steps=1, opt_steps=1, refine_steps=0, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-09-02 13:46:25.210] [info] Group small size: 0
[2025-09-02 13:46:25.210] [info] Group medium size: 0
[2025-09-02 13:46:25.210] [info] Group large size: 1
[2025-09-02 13:46:25.210] [info] Group extra size: 0
[2025-09-02 13:46:25.210] [info] Group overflow size: 0
[2025-09-02 13:46:25.210] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@@@@@@

In [15]:
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align/align_4IEH/same_ring/"
fn_pdb = "4IEH.pdb"
fn_sdf_q = "CHEMBL376408_prepared.sdf"
fn_sdf_t = "CHEMBL2322026_8.0_nM_3d_dock.sdf"
center = [12, 25, 11]

fn_sdf_q_rand = generate_rand(dp, fn_pdb, fn_sdf_q, center)
fn_sdf_q = fn_sdf_q_rand
mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 4096, 0.3)



    ██╗   ██╗██████╗ ██████╗
    ██║   ██║██╔══██╗╚════██╗
    ██║   ██║██║  ██║ █████╔╝
    ██║   ██║██║  ██║██╔═══╝
    ╚██████╔╝██████╔╝███████╗
     ╚═════╝ ╚═════╝ ╚══════╝

    DP Technology Docking Toolkit


{'receptor': '4IEH.pdb', 'ligand': 'CHEMBL376408_prepared.sdf', 'ligand_batch': None, 'center': [12.0, 25.0, 11.0], 'exhaustiveness': 10, 'randomize': True, 'mc_steps': 1, 'opt_steps': 1, 'refine_steps': 0, 'num_pose': 1, 'rmsd_limit': 1.0, 'energy_range': 3.0, 'seed': 12345, 'use_tor_lib': False, 'n_cpu': None, 'gpu_device_id': 0, 'box_size': [30.0, 30.0, 30.0], 'task': 'screen', 'search_mode': 'free', 'template_docking': False, 'reference_sdf_file_name': None, 'compute_center': True, 'core_atom_mapping_dict_list': None, 'covalent_ligand': False, 'covalent_residue_atom_info_list': None, 'preserve_receptor_hydrogen': False, 'temp_dir_name': './tmp', 'output_receptor_dms_file_name': 'receptor_parameterized.dms', 'output_docking_pose_sdf_file_name': 'CHEMBL376408_prepared_ra

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:442: UserWarning: PDB file contained CONECT record to TER entry. These are not included in bonds.
  warnings.warn(
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-09-01 16:13:48.092] [info] ----------------------- RUN Screening -----------------------
[2025-09-01 16:13:48.092] [info] Receptor loaded: 1100 atoms in box
[2025-09-01 16:13:48.093] [info] Ligands loaded. Total count: 1
[2025-09-01 16:13:48.154] [info] ----------------------- RUN Screening -----------------------
[2025-09-01 16:13:48.154] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=-3 Angstrom, x_hi=27 Angstrom, y_lo=10 Angstrom, y_hi=40 Angstrom, z_lo=-4 Angstrom, z_hi=26 Angstrom, 
constraint_docking=false, exhaustiveness=10, 
mc_steps=1, opt_steps=1, refine_steps=0, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-09-01 16:13:48.154] [info] Group small size: 0
[2025-09-01 16:13:48.154] [info] Group medium size: 0
[2025-09-01 16:13:48.154] [info] Group large size: 1
[2025-09-01 16:13:48.154] [info] Group extra size: 0
[2025-09-01 16:13:48.154] [info] Group overflow size: 0
[2025-09-01 16:13:48.154] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@@@@@@

### Bace

In [47]:
dp_Bace = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace_Chiral"
fn_pdb = "Bace.pdb"
fn_sdf_q = "0.sdf"
fn_sdf_t = "1.sdf"
center = [15, -1, -1]

for i in [0, 2, 3, 4]:
    fn_sdf_q = f"{i}.sdf"
    dp = os.path.join(dp_Bace, f"1_{i}")
    
    fn_sdf_q_rand = generate_rand(dp, fn_pdb, fn_sdf_q, center)
    fn_sdf_q = fn_sdf_q_rand
    mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 512, 0.3)



    ██╗   ██╗██████╗ ██████╗
    ██║   ██║██╔══██╗╚════██╗
    ██║   ██║██║  ██║ █████╔╝
    ██║   ██║██║  ██║██╔═══╝
    ╚██████╔╝██████╔╝███████╗
     ╚═════╝ ╚═════╝ ╚══════╝

    DP Technology Docking Toolkit




{'receptor': 'Bace.pdb', 'ligand': '0.sdf', 'ligand_batch': None, 'center': [15.0, -1.0, -1.0], 'exhaustiveness': 10, 'randomize': True, 'mc_steps': 1, 'opt_steps': 1, 'refine_steps': 0, 'num_pose': 1, 'rmsd_limit': 1.0, 'energy_range': 3.0, 'seed': 12345, 'use_tor_lib': False, 'n_cpu': None, 'gpu_device_id': 0, 'box_size': [30.0, 30.0, 30.0], 'task': 'screen', 'search_mode': 'free', 'template_docking': False, 'reference_sdf_file_name': None, 'compute_center': True, 'core_atom_mapping_dict_list': None, 'covalent_ligand': False, 'covalent_residue_atom_info_list': None, 'preserve_receptor_hydrogen': False, 'temp_dir_name': './tmp', 'output_receptor_dms_file_name': 'receptor_parameterized.dms', 'output_docking_pose_sdf_file_name': '0_rand.sdf'}
Target Center for Current Docking: (15.0, -1.0, -1.0)


/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


[2025-08-29 13:19:23.721] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:19:23.721] [info] Receptor loaded: 2393 atoms in box
[2025-08-29 13:19:23.722] [info] Ligands loaded. Total count: 1
[2025-08-29 13:19:23.780] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:19:23.780] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=0 Angstrom, x_hi=30 Angstrom, y_lo=-16 Angstrom, y_hi=14 Angstrom, z_lo=-16 Angstrom, z_hi=14 Angstrom, 
constraint_docking=false, exhaustiveness=10, 
mc_steps=1, opt_steps=1, refine_steps=0, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-29 13:19:23.780] [info] Group small size: 0
[2025-08-29 13:19:23.780] [info] Group medium size: 1
[2025-08-29 13:19:23.780] [info] Group large size: 0
[2025-08-29 13:19:23.780] [info] Group extra size: 0
[2025-08-29 13:19:23.780] [info] Group overflow size: 0
[2025-08-29 13:19:23.780] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@@@@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


...

[2025-08-29 13:24:24.531] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:24:24.531] [info] Receptor loaded: 2393 atoms in box
[2025-08-29 13:24:24.532] [info] Ligands loaded. Total count: 1
[2025-08-29 13:24:24.590] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:24:24.590] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=0 Angstrom, x_hi=30 Angstrom, y_lo=-16 Angstrom, y_hi=14 Angstrom, z_lo=-16 Angstrom, z_hi=14 Angstrom, 
constraint_docking=false, exhaustiveness=10, 
mc_steps=1, opt_steps=1, refine_steps=0, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-29 13:24:24.590] [info] Group small size: 0
[2025-08-29 13:24:24.590] [info] Group medium size: 1
[2025-08-29 13:24:24.590] [info] Group large size: 0
[2025-08-29 13:24:24.590] [info] Group extra size: 0
[2025-08-29 13:24:24.590] [info] Group overflow size: 0
[2025-08-29 13:24:24.590] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@@@@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


[2025-08-29 13:24:55.143] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:24:55.143] [info] Receptor loaded: 2390 atoms in box
[2025-08-29 13:24:55.144] [info] Ligands loaded. Total count: 1
[2025-08-29 13:24:55.207] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:24:55.207] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=0.0044555664 Angstrom, x_hi=30.004456 Angstrom, y_lo=-16.513166 Angstrom, y_hi=13.486834 Angstrom, z_lo=-15.813394 Angstrom, z_hi=14.186606 Angstrom, 
constraint_docking=true, exhaustiveness=512, 
mc_steps=40, opt_steps=-1, refine_steps=5, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-29 13:24:55.207] [info] Group small size: 0
[2025-08-29 13:24:55.207] [info] Group medium size: 1
[2025-08-29 13:24:55.207] [info] Group large size: 0
[2025-08-29 13:24:55.207] [info] Group extra size: 0
[2025-08-29 13:24:55.207] [info] Group overflow size: 0
[2025-08-29 13:24:55.207] [info] @@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


[2025-08-29 13:25:26.027] [info] ----------------------- RUN Only Scoring -----------------------
[2025-08-29 13:25:26.027] [info] Receptor loaded: 2393 atoms in box
[2025-08-29 13:25:26.028] [info] Ligands loaded. Total count: 1
[2025-08-29 13:25:26.088] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:25:26.088] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=0 Angstrom, x_hi=30 Angstrom, y_lo=-16 Angstrom, y_hi=14 Angstrom, z_lo=-16 Angstrom, z_hi=14 Angstrom, 
constraint_docking=false, exhaustiveness=1, 
mc_steps=0, opt_steps=0, refine_steps=0, 
num_pose=1, rmsd_limit=999 Angstrom, 

[2025-08-29 13:25:26.088] [info] Group small size: 0
[2025-08-29 13:25:26.088] [info] Group medium size: 1
[2025-08-29 13:25:26.088] [info] Group large size: 0
[2025-08-29 13:25:26.088] [info] Group extra size: 0
[2025-08-29 13:25:26.088] [info] Group overflow size: 0
[2025-08-29 13:25:26.088] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@

### HNE

In [46]:
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_HNE"
fn_pdb = "5A0A_20.pdb"
fn_sdf_q = "HNE_29.sdf"
fn_sdf_t = "HNE_Ref.sdf"
center = [-7, 35, -3]

fn_sdf_q_rand = generate_rand(dp, fn_pdb, fn_sdf_q, center)
fn_sdf_q = fn_sdf_q_rand
mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 512, 0.3)



    ██╗   ██╗██████╗ ██████╗
    ██║   ██║██╔══██╗╚════██╗
    ██║   ██║██║  ██║ █████╔╝
    ██║   ██║██║  ██║██╔═══╝
    ╚██████╔╝██████╔╝███████╗
     ╚═════╝ ╚═════╝ ╚══════╝

    DP Technology Docking Toolkit


{'receptor': '5A0A_20.pdb', 'ligand': 'HNE_29.sdf', 'ligand_batch': None, 'center': [-7.0, 35.0, -3.0], 'exhaustiveness': 10, 'randomize': True, 'mc_steps': 1, 'opt_steps': 1, 'refine_steps': 0, 'num_pose': 1, 'rmsd_limit': 1.0, 'energy_range': 3.0, 'seed': 12345, 'use_tor_lib': False, 'n_cpu': None, 'gpu_device_id': 0, 'box_size': [30.0, 30.0, 30.0], 'task': 'screen', 'search_mode': 'free', 'template_docking': False, 'reference_sdf_file_name': None, 'compute_center': True, 'core_atom_mapping_dict_list': None, 'covalent_ligand': False, 'covalent_residue_atom_info_list': None, 'preserve_receptor_hydrogen': False, 'temp_dir_name': './tmp', 'output_receptor_dms_file_name': 'receptor_parameterized.dms', 'output_docking_pose_sdf_file_name': 'HNE_29_rand.sdf'}
Target Center for 

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


[2025-08-29 13:18:24.840] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:18:24.840] [info] Receptor loaded: 1471 atoms in box
[2025-08-29 13:18:24.841] [info] Ligands loaded. Total count: 1
[2025-08-29 13:18:24.899] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:18:24.899] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=-22 Angstrom, x_hi=8 Angstrom, y_lo=20 Angstrom, y_hi=50 Angstrom, z_lo=-18 Angstrom, z_hi=12 Angstrom, 
constraint_docking=false, exhaustiveness=10, 
mc_steps=1, opt_steps=1, refine_steps=0, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-29 13:18:24.899] [info] Group small size: 0
[2025-08-29 13:18:24.899] [info] Group medium size: 1
[2025-08-29 13:18:24.899] [info] Group large size: 0
[2025-08-29 13:18:24.899] [info] Group extra size: 0
[2025-08-29 13:18:24.899] [info] Group overflow size: 0
[2025-08-29 13:18:24.899] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@@@@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/unidock_processing/ligand_topology/utils.py:169: UserWarning: Number of neighbor atoms does not match between query and reference atoms! Please look at this case carefully!
  warnings.warn(


[2025-08-29 13:18:31.449] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:18:31.449] [info] Receptor loaded: 1471 atoms in box
[2025-08-29 13:18:31.449] [info] Ligands loaded. Total count: 1
[2025-08-29 13:18:31.516] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:18:31.516] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=-22.088043 Angstrom, x_hi=7.9119563 Angstrom, y_lo=20.082752 Angstrom, y_hi=50.082752 Angstrom, z_lo=-18.072794 Angstrom, z_hi=11.927207 Angstrom, 
constraint_docking=true, exhaustiveness=512, 
mc_steps=40, opt_steps=-1, refine_steps=5, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-29 13:18:31.516] [info] Group small size: 0
[2025-08-29 13:18:31.516] [info] Group medium size: 1
[2025-08-29 13:18:31.516] [info] Group large size: 0
[2025-08-29 13:18:31.516] [info] Group extra size: 0
[2025-08-29 13:18:31.516] [info] Group overflow size: 0
[2025-08-29 13:18:31.516] [info] @@@@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


[2025-08-29 13:18:46.287] [info] ----------------------- RUN Only Scoring -----------------------
[2025-08-29 13:18:46.287] [info] Receptor loaded: 1471 atoms in box
[2025-08-29 13:18:46.288] [info] Ligands loaded. Total count: 1
[2025-08-29 13:18:46.353] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:18:46.353] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=-22 Angstrom, x_hi=8 Angstrom, y_lo=20 Angstrom, y_hi=50 Angstrom, z_lo=-18 Angstrom, z_hi=12 Angstrom, 
constraint_docking=false, exhaustiveness=1, 
mc_steps=0, opt_steps=0, refine_steps=0, 
num_pose=1, rmsd_limit=999 Angstrom, 

[2025-08-29 13:18:46.353] [info] Group small size: 0
[2025-08-29 13:18:46.353] [info] Group medium size: 1
[2025-08-29 13:18:46.353] [info] Group large size: 0
[2025-08-29 13:18:46.353] [info] Group extra size: 0
[2025-08-29 13:18:46.353] [info] Group overflow size: 0
[2025-08-29 13:18:46.353] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@

### Cyclic Peptide

In [45]:
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_cyclic_peptide"
fn_pdb = "protein.pdb"
fn_sdf_q = "78_1_5.sdf"
fn_sdf_t = "56_35_2.sdf"
center = [20, -1, 85]

fn_sdf_q_rand = generate_rand(dp, fn_pdb, fn_sdf_q, center)
fn_sdf_q = fn_sdf_q_rand
mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 512, 0.3)



    ██╗   ██╗██████╗ ██████╗
    ██║   ██║██╔══██╗╚════██╗
    ██║   ██║██║  ██║ █████╔╝
    ██║   ██║██║  ██║██╔═══╝
    ╚██████╔╝██████╔╝███████╗
     ╚═════╝ ╚═════╝ ╚══════╝

    DP Technology Docking Toolkit


{'receptor': 'protein.pdb', 'ligand': '78_1_5.sdf', 'ligand_batch': None, 'center': [20.0, -1.0, 85.0], 'exhaustiveness': 10, 'randomize': True, 'mc_steps': 1, 'opt_steps': 1, 'refine_steps': 0, 'num_pose': 1, 'rmsd_limit': 1.0, 'energy_range': 3.0, 'seed': 12345, 'use_tor_lib': False, 'n_cpu': None, 'gpu_device_id': 0, 'box_size': [30.0, 30.0, 30.0], 'task': 'screen', 'search_mode': 'free', 'template_docking': False, 'reference_sdf_file_name': None, 'compute_center': True, 'core_atom_mapping_dict_list': None, 'covalent_ligand': False, 'covalent_residue_atom_info_list': None, 'preserve_receptor_hydrogen': False, 'temp_dir_name': './tmp', 'output_receptor_dms_file_name': 'receptor_parameterized.dms', 'output_docking_pose_sdf_file_name': '78_1_5_rand.sdf'}
Target Center for 

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-08-29 13:15:32.166] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:15:32.167] [info] Receptor loaded: 1172 atoms in box
[2025-08-29 13:15:32.168] [info] Ligands loaded. Total count: 1
[2025-08-29 13:15:32.232] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:15:32.232] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=5 Angstrom, x_hi=35 Angstrom, y_lo=-16 Angstrom, y_hi=14 Angstrom, z_lo=70 Angstrom, z_hi=100 Angstrom, 
constraint_docking=false, exhaustiveness=10, 
mc_steps=1, opt_steps=1, refine_steps=0, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-29 13:15:32.232] [info] Group small size: 0
[2025-08-29 13:15:32.232] [info] Group medium size: 0
[2025-08-29 13:15:32.232] [info] Group large size: 0
[2025-08-29 13:15:32.232] [info] Group extra size: 0
[2025-08-29 13:15:32.232] [info] Group overflow size: 1
[2025-08-29 13:15:32.232] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@@@@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-08-29 13:17:29.860] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:17:29.860] [info] Receptor loaded: 1191 atoms in box
[2025-08-29 13:17:29.862] [info] Ligands loaded. Total count: 1
[2025-08-29 13:17:29.920] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:17:29.920] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=4.7995014 Angstrom, x_hi=34.7995 Angstrom, y_lo=-16.11083 Angstrom, y_hi=13.889169 Angstrom, z_lo=69.680115 Angstrom, z_hi=99.680115 Angstrom, 
constraint_docking=true, exhaustiveness=512, 
mc_steps=40, opt_steps=-1, refine_steps=5, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-29 13:17:29.920] [info] Group small size: 0
[2025-08-29 13:17:29.920] [info] Group medium size: 0
[2025-08-29 13:17:29.920] [info] Group large size: 0
[2025-08-29 13:17:29.920] [info] Group extra size: 0
[2025-08-29 13:17:29.920] [info] Group overflow size: 1
[2025-08-29 13:17:29.920] [info] @@@@@@@@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-08-29 13:17:44.246] [info] ----------------------- RUN Only Scoring -----------------------
[2025-08-29 13:17:44.246] [info] Receptor loaded: 1172 atoms in box
[2025-08-29 13:17:44.248] [info] Ligands loaded. Total count: 1
[2025-08-29 13:17:44.313] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:17:44.313] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=5 Angstrom, x_hi=35 Angstrom, y_lo=-16 Angstrom, y_hi=14 Angstrom, z_lo=70 Angstrom, z_hi=100 Angstrom, 
constraint_docking=false, exhaustiveness=1, 
mc_steps=0, opt_steps=0, refine_steps=0, 
num_pose=1, rmsd_limit=999 Angstrom, 

[2025-08-29 13:17:44.313] [info] Group small size: 0
[2025-08-29 13:17:44.313] [info] Group medium size: 0
[2025-08-29 13:17:44.313] [info] Group large size: 0
[2025-08-29 13:17:44.313] [info] Group extra size: 0
[2025-08-29 13:17:44.313] [info] Group overflow size: 1
[2025-08-29 13:17:44.313] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@

### 2ZP0

In [44]:
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_2ZP0/"
fn_pdb = "2ZP0.pdb"
fn_sdf_q = "CHEMBL498640_5000.0_nM_3d_dock.sdf"
fn_sdf_t = "CHEMBL498339_42.0_nM_3d_dock_tpl.sdf"
center = [33, 6, 10]

fn_sdf_q_rand = generate_rand(dp, fn_pdb, fn_sdf_q, center)
fn_sdf_q = fn_sdf_q_rand
mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 512, 0.3)



    ██╗   ██╗██████╗ ██████╗
    ██║   ██║██╔══██╗╚════██╗
    ██║   ██║██║  ██║ █████╔╝
    ██║   ██║██║  ██║██╔═══╝
    ╚██████╔╝██████╔╝███████╗
     ╚═════╝ ╚═════╝ ╚══════╝

    DP Technology Docking Toolkit




{'receptor': '2ZP0.pdb', 'ligand': 'CHEMBL498640_5000.0_nM_3d_dock.sdf', 'ligand_batch': None, 'center': [33.0, 6.0, 10.0], 'exhaustiveness': 10, 'randomize': True, 'mc_steps': 1, 'opt_steps': 1, 'refine_steps': 0, 'num_pose': 1, 'rmsd_limit': 1.0, 'energy_range': 3.0, 'seed': 12345, 'use_tor_lib': False, 'n_cpu': None, 'gpu_device_id': 0, 'box_size': [30.0, 30.0, 30.0], 'task': 'screen', 'search_mode': 'free', 'template_docking': False, 'reference_sdf_file_name': None, 'compute_center': True, 'core_atom_mapping_dict_list': None, 'covalent_ligand': False, 'covalent_residue_atom_info_list': None, 'preserve_receptor_hydrogen': False, 'temp_dir_name': './tmp', 'output_receptor_dms_file_name': 'receptor_parameterized.dms', 'output_docking_pose_sdf_file_name': 'CHEMBL498640_5000.0_nM_3d_dock_rand.sdf'}
Target Center for Current Docking: (33.0, 6.0, 10.0)


/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:442: UserWarning: PDB file contained CONECT record to TER entry. These are not included in bonds.
  warnings.warn(
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-08-29 13:14:46.165] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:14:46.165] [info] Receptor loaded: 1627 atoms in box
[2025-08-29 13:14:46.166] [info] Ligands loaded. Total count: 1
[2025-08-29 13:14:46.228] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:14:46.228] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=18 Angstrom, x_hi=48 Angstrom, y_lo=-9 Angstrom, y_hi=21 Angstrom, z_lo=-5 Angstrom, z_hi=25 Angstrom, 
constraint_docking=false, exhaustiveness=10, 
mc_steps=1, opt_steps=1, refine_steps=0, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-29 13:14:46.228] [info] Group small size: 0
[2025-08-29 13:14:46.228] [info] Group medium size: 1
[2025-08-29 13:14:46.228] [info] Group large size: 0
[2025-08-29 13:14:46.228] [info] Group extra size: 0
[2025-08-29 13:14:46.228] [info] Group overflow size: 0
[2025-08-29 13:14:46.228] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@@@@@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:442: UserWarning: PDB file contained CONECT record to TER entry. These are not included in bonds.
  warnings.warn(
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-08-29 13:14:56.023] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:14:56.023] [info] Receptor loaded: 1606 atoms in box
[2025-08-29 13:14:56.023] [info] Ligands loaded. Total count: 1
[2025-08-29 13:14:56.084] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:14:56.084] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=18.586464 Angstrom, x_hi=48.586464 Angstrom, y_lo=-8.997392 Angstrom, y_hi=21.00261 Angstrom, z_lo=-4.7904987 Angstrom, z_hi=25.209501 Angstrom, 
constraint_docking=true, exhaustiveness=512, 
mc_steps=40, opt_steps=-1, refine_steps=5, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-29 13:14:56.084] [info] Group small size: 0
[2025-08-29 13:14:56.084] [info] Group medium size: 1
[2025-08-29 13:14:56.084] [info] Group large size: 0
[2025-08-29 13:14:56.084] [info] Group extra size: 0
[2025-08-29 13:14:56.084] [info] Group overflow size: 0
[2025-08-29 13:14:56.084] [info] @@@@@@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:442: UserWarning: PDB file contained CONECT record to TER entry. These are not included in bonds.
  warnings.warn(
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-08-29 13:15:12.512] [info] ----------------------- RUN Only Scoring -----------------------
[2025-08-29 13:15:12.512] [info] Receptor loaded: 1627 atoms in box
[2025-08-29 13:15:12.513] [info] Ligands loaded. Total count: 1
[2025-08-29 13:15:12.579] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 13:15:12.579] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=18 Angstrom, x_hi=48 Angstrom, y_lo=-9 Angstrom, y_hi=21 Angstrom, z_lo=-5 Angstrom, z_hi=25 Angstrom, 
constraint_docking=false, exhaustiveness=1, 
mc_steps=0, opt_steps=0, refine_steps=0, 
num_pose=1, rmsd_limit=999 Angstrom, 

[2025-08-29 13:15:12.579] [info] Group small size: 0
[2025-08-29 13:15:12.579] [info] Group medium size: 1
[2025-08-29 13:15:12.579] [info] Group large size: 0
[2025-08-29 13:15:12.579] [info] Group extra size: 0
[2025-08-29 13:15:12.579] [info] Group overflow size: 0
[2025-08-29 13:15:12.579] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@@

### 3VOM

In [61]:
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_3OVM/ref"
fn_pdb = "3OVM.pdb"
fn_sdf_q = "CHEMBL458209_21.0_nM_3d_dock.sdf"
fn_sdf_t = "ref.sdf" #"CHEMBL504585_58.0_nM_3d_dock.sdf"
center = [-5, -5, -15]

fn_sdf_q_rand = generate_rand(dp, fn_pdb, fn_sdf_q, center)
fn_sdf_q = fn_sdf_q_rand
mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 512, 0.3)



    ██╗   ██╗██████╗ ██████╗
    ██║   ██║██╔══██╗╚════██╗
    ██║   ██║██║  ██║ █████╔╝
    ██║   ██║██║  ██║██╔═══╝
    ╚██████╔╝██████╔╝███████╗
     ╚═════╝ ╚═════╝ ╚══════╝

    DP Technology Docking Toolkit


{'receptor': '3OVM.pdb', 'ligand': 'CHEMBL458209_21.0_nM_3d_dock.sdf', 'ligand_batch': None, 'center': [-5.0, -5.0, -15.0], 'exhaustiveness': 10, 'randomize': True, 'mc_steps': 1, 'opt_steps': 1, 'refine_steps': 0, 'num_pose': 1, 'rmsd_limit': 1.0, 'energy_range': 3.0, 'seed': 12345, 'use_tor_lib': False, 'n_cpu': None, 'gpu_device_id': 0, 'box_size': [30.0, 30.0, 30.0], 'task': 'screen', 'search_mode': 'free', 'template_docking': False, 'reference_sdf_file_name': None, 'compute_center': True, 'core_atom_mapping_dict_list': None, 'covalent_ligand': False, 'covalent_residue_atom_info_list': None, 'preserve_receptor_hydrogen': False, 'temp_dir_name': './tmp', 'output_receptor_dms_file_name': 'receptor_parameterized.dms', 'output_docking_pose_sdf_file_name': 'CHEMBL458209_21.

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:442: UserWarning: PDB file contained CONECT record to TER entry. These are not included in bonds.
  warnings.warn(
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-08-29 15:54:12.490] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 15:54:12.490] [info] Receptor loaded: 2149 atoms in box
[2025-08-29 15:54:12.491] [info] Ligands loaded. Total count: 1
[2025-08-29 15:54:12.555] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 15:54:12.555] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=-20 Angstrom, x_hi=10 Angstrom, y_lo=-20 Angstrom, y_hi=10 Angstrom, z_lo=-30 Angstrom, z_hi=0 Angstrom, 
constraint_docking=false, exhaustiveness=10, 
mc_steps=1, opt_steps=1, refine_steps=0, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-29 15:54:12.555] [info] Group small size: 1
[2025-08-29 15:54:12.555] [info] Group medium size: 0
[2025-08-29 15:54:12.555] [info] Group large size: 0
[2025-08-29 15:54:12.555] [info] Group extra size: 0
[2025-08-29 15:54:12.555] [info] Group overflow size: 0
[2025-08-29 15:54:12.555] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@@@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:442: UserWarning: PDB file contained CONECT record to TER entry. These are not included in bonds.
  warnings.warn(
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/unidock_processing/ligand_topology/utils.py:169: UserWarning: Number of neighbor atoms does not match between query and reference atoms! Please look at this case carefully!
  warnings.warn(


[2025-08-29 15:54:27.156] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 15:54:27.156] [info] Receptor loaded: 2175 atoms in box
[2025-08-29 15:54:27.157] [info] Ligands loaded. Total count: 1
[2025-08-29 15:54:27.219] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 15:54:27.219] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=-19.624836 Angstrom, x_hi=10.375164 Angstrom, y_lo=-20.243652 Angstrom, y_hi=9.756349 Angstrom, z_lo=-30.696869 Angstrom, z_hi=-0.69686794 Angstrom, 
constraint_docking=true, exhaustiveness=512, 
mc_steps=40, opt_steps=-1, refine_steps=5, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-29 15:54:27.219] [info] Group small size: 1
[2025-08-29 15:54:27.219] [info] Group medium size: 0
[2025-08-29 15:54:27.219] [info] Group large size: 0
[2025-08-29 15:54:27.219] [info] Group extra size: 0
[2025-08-29 15:54:27.219] [info] Group overflow size: 0
[2025-08-29 15:54:27.219] [info] @@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:442: UserWarning: PDB file contained CONECT record to TER entry. These are not included in bonds.
  warnings.warn(
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-08-29 15:54:46.639] [info] ----------------------- RUN Only Scoring -----------------------
[2025-08-29 15:54:46.639] [info] Receptor loaded: 2149 atoms in box
[2025-08-29 15:54:46.639] [info] Ligands loaded. Total count: 1
[2025-08-29 15:54:46.705] [info] ----------------------- RUN Screening -----------------------
[2025-08-29 15:54:46.705] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=-20 Angstrom, x_hi=10 Angstrom, y_lo=-20 Angstrom, y_hi=10 Angstrom, z_lo=-30 Angstrom, z_hi=0 Angstrom, 
constraint_docking=false, exhaustiveness=1, 
mc_steps=0, opt_steps=0, refine_steps=0, 
num_pose=1, rmsd_limit=999 Angstrom, 

[2025-08-29 15:54:46.705] [info] Group small size: 1
[2025-08-29 15:54:46.705] [info] Group medium size: 0
[2025-08-29 15:54:46.705] [info] Group large size: 0
[2025-08-29 15:54:46.705] [info] Group extra size: 0
[2025-08-29 15:54:46.705] [info] Group overflow size: 0
[2025-08-29 15:54:46.705] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@

### multi 7KK6

In [32]:
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align/align_multi_7KK6"
fn_pdb = "7KK6.pdb"
fn_sdf_q = "small.sdf"
fn_sdf_t = "ref.sdf"

center = Chem.SDMolSupplier(os.path.join(dp, fn_sdf_t), removeHs=False)[0].GetConformer().GetPositions().mean(axis=0).tolist()

fn_sdf_q_rand = generate_rand(dp, fn_pdb, fn_sdf_q, center)
fn_sdf_q = fn_sdf_q_rand
mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 512, 0.02, 1)



    ██╗   ██╗██████╗ ██████╗
    ██║   ██║██╔══██╗╚════██╗
    ██║   ██║██║  ██║ █████╔╝
    ██║   ██║██║  ██║██╔═══╝
    ╚██████╔╝██████╔╝███████╗
     ╚═════╝ ╚═════╝ ╚══════╝

    DP Technology Docking Toolkit


{'receptor': '7KK6.pdb', 'ligand': 'small.sdf', 'ligand_batch': None, 'center': [2.005471428571429, -17.452677142857144, 34.19988285714287], 'exhaustiveness': 10, 'randomize': True, 'mc_steps': 1, 'opt_steps': 1, 'refine_steps': 0, 'num_pose': 1, 'rmsd_limit': 1.0, 'energy_range': 3.0, 'seed': 12345, 'use_tor_lib': False, 'n_cpu': None, 'gpu_device_id': 0, 'box_size': [30.0, 30.0, 30.0], 'task': 'screen', 'search_mode': 'free', 'template_docking': False, 'reference_sdf_file_name': None, 'compute_center': True, 'core_atom_mapping_dict_list': None, 'covalent_ligand': False, 'covalent_residue_atom_info_list': None, 'preserve_receptor_hydrogen': False, 'temp_dir_name': './tmp', 'output_receptor_dms_file_name': 'receptor_parameterized.dms', 'output_docking_pose_sdf_file_name': 

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:442: UserWarning: PDB file contained CONECT record to TER entry. These are not included in bonds.
  warnings.warn(
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-09-02 16:43:38.009] [info] ----------------------- RUN Screening -----------------------
[2025-09-02 16:43:38.010] [info] Receptor loaded: 2333 atoms in box
[2025-09-02 16:43:38.013] [info] Ligands loaded. Total count: 8
[2025-09-02 16:43:38.070] [info] ----------------------- RUN Screening -----------------------
[2025-09-02 16:43:38.070] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=-12.994529 Angstrom, x_hi=17.005472 Angstrom, y_lo=-32.452675 Angstrom, y_hi=-2.4526768 Angstrom, z_lo=19.199883 Angstrom, z_hi=49.199883 Angstrom, 
constraint_docking=false, exhaustiveness=10, 
mc_steps=1, opt_steps=1, refine_steps=0, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-09-02 16:43:38.070] [info] Group small size: 2
[2025-09-02 16:43:38.070] [info] Group medium size: 6
[2025-09-02 16:43:38.070] [info] Group large size: 0
[2025-09-02 16:43:38.070] [info] Group extra size: 0
[2025-09-02 16:43:38.070] [info] Group overflow size: 0
[2025-09-02 16:43:38.070] [info] @@@@@@

# Others

## 查看分子结构图

In [7]:

from IPython.display import SVG
from rdkit.Chem.Draw import rdMolDraw2D
from copy import deepcopy

def show_atom_number(mol, label='atomNote'):
    new_mol = deepcopy(mol)
    for atom in new_mol.GetAtoms():
        atom.SetProp(label, str(atom.GetIdx()))
    return new_mol
def draw_mol(mol, highlight_atoms=[], molSize=(600, 400), kekulize=True):
    drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0], molSize[1])
    drawer.drawOptions().addAtomIndices = True
    drawer.DrawMolecule(mol, highlightAtoms=highlight_atoms)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    return SVG(svg)

from IPython.display import HTML

def draw_mols(mols: list, highlight_atoms=None, width=300):
    """显示多个分子（横向排列）"""
    if highlight_atoms is None:
        highlight_atoms = [[] for _ in mols]
    
    if not isinstance(width, list):
        width = [width] * len(mols)
    # 生成独立svg
    list_svg = [draw_mol(mol, highlight_atoms=hl, molSize=(w, 400)).data for mol, hl, w in zip(mols, highlight_atoms, width)]
    
    # 创建横向布局HTML
    html_code = f"""
    <div style="display: flex; gap: 20px;">"""
    html_code += "".join([f"<div>{svg.replace('width:600px', f'width:{width}px')}</div>" for svg in list_svg])
    html_code += "</div>"
    return HTML(html_code)


In [13]:
def read_sdf(fp_sdf):
    m1 = Chem.SDMolSupplier(fp_sdf, removeHs=True)[0]
    m1.RemoveAllConformers()
    return m1

In [ ]:
m1 = read_sdf("HNE_Ref.sdf")
m2 = read_sdf("HNE_2.sdf")
draw_mols([m1, m2], width=600)

## Transfer .MOL file to .SDF File

In [22]:
def transfer_mol_to_sdf(mol_file_path):
    fp_sdf = mol_file_path.replace(".mol", ".sdf")
    mol = Chem.MolFromMolFile(mol_file_path)
    Chem.MolToMolFile(mol, fp_sdf)
    return fp_sdf

transfer_mol_to_sdf("/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/test_all_rings.mol")

'/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/test_all_rings.sdf'

## 查看JSON里的torsion数量

In [9]:
fp_json = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align/align_multi_7KK6/ud2_align.json"
with open(fp_json, "r") as f:
    data = json.load(f)


len([i[0] for i in data["MOL_0"]["torsions"]])

7

In [10]:
data.keys()

dict_keys(['receptor', 'MOL_0', 'MOL_1', 'MOL_2', 'MOL_3', 'MOL_4', 'MOL_5', 'MOL_6', 'MOL_7', 'score'])

# 自动流程筛选测试cases

## Get Cases from Rongfeng HuoShan

In [ ]:
# Collect all cases
import os
import shutil
dp_res = os.path.abspath("all_data_for_align")
if os.path.exists(dp_res):
    shutil.rmtree(dp_res)
for d in os.listdir("."):
    if not d.startswith("CHEMBL"):
        continue
    if not os.path.isdir(d):
        continue
    list_fn = os.listdir(d)
    fn_pdb = [i for i in list_fn if i.endswith(".pdb")][0]
    fn_dms = "receptor_parameterized.dms"
    list_fn_sdf = [i for i in list_fn if i.endswith(".sdf")]
    list_fn_ref = [i for i in list_fn_sdf if "ref" in i]
    list_fn_sdf_raw = [i for i in list_fn_sdf if not (("3d_dock" in i) or ("align" in i))]
    # copy to res
    dp_res_case = os.path.join(dp_res, d)
    os.makedirs(dp_res_case, exist_ok=True)
    if not os.path.exists(os.path.join(d, fn_dms)):
        continue
    shutil.copy(os.path.join(d, fn_pdb), os.path.join(dp_res_case, fn_pdb))
    shutil.copy(os.path.join(d, fn_dms), os.path.join(dp_res_case, fn_dms))
    for fn in list_fn_ref:
        shutil.copy(os.path.join(d, fn), os.path.join(dp_res_case, fn))
    for fn in list_fn_sdf_raw:
        shutil.copy(os.path.join(d, fn), os.path.join(dp_res_case, fn))

### Score all template sdf

In [2]:
# perform scoring
import os
import subprocess as sp
import time
import yaml
from rdkit import Chem

def read_sdf(fp_sdf):
    m1 = Chem.SDMolSupplier(fp_sdf, removeHs=False)[0]
    return m1

def generate_score(dp:str):

    # time_start = time.time()
    try:
        # ------------- Read file path info
        fn_ref = [i for i in os.listdir(dp) if i.endswith("_ref.sdf")][0]
        fp_ref = os.path.join(dp, fn_ref)
        m_ref = Chem.SDMolSupplier(fp_ref, removeHs=False)[0]
        center = m_ref.GetConformer().GetPositions().mean(axis=0).tolist()

        # ------------- 执行 unidock2 docking
        fp_config_unidock2 = os.path.join(dp, "unidock2_score.yaml")
        config_score = get_config("free")
        config_score["Settings"]["task"] = "score"
        
        fn_out = fn_ref[:-4] + "_score.sdf"
        config_score["Preprocessing"]["output_docking_pose_sdf_file_name"] = fn_out
        

        config_score["Required"]["receptor"] = "receptor_parameterized.dms"
        config_score["Required"]["ligand"] = fn_ref
        config_score["Required"]["center"] = center
        
        with open(fp_config_unidock2, "w") as f:
            yaml.dump(config_score, f)

        with open(os.path.join(dp, "all.log"), "a") as log_file:
            sp.run(f"unidock2 docking -cf {fp_config_unidock2}", shell=True, cwd=dp, stdout=log_file, stderr=log_file)


        # time_end = time.time()
        # print(f"\n\nScoring cost: {time_end - time_start} seconds\n\n")
        return fn_out
    
    except Exception as e:
        print(f"发生异常: {e}")
        return "FAILED"
    

In [ ]:
dp_all = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align/batch/all_data_for_align"

from multiprocessing import Pool
from tqdm import tqdm

if __name__ == "__main__":
    dirs = [os.path.join(dp_all, d) for d in os.listdir(dp_all)]
    with Pool(16) as pool:
        imap_it = pool.imap(generate_score, dirs)
        for _ in tqdm(imap_it, total=len(dirs), desc="Scoring"):
            pass
    


发生异常: list index out of range


Scoring:   0%|          | 0/1243 [00:00<?, ?it/s]

Scoring:   1%|          | 7/1243 [00:45<2:17:44,  6.69s/it]

发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list

Scoring:   1%|          | 9/1243 [11:57<40:06:01, 116.99s/it]

发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range


Scoring:   6%|▌         | 69/1243 [19:07<4:40:09, 14.32s/it] 

发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range


Scoring:  32%|███▏      | 400/1243 [20:26<26:48,  1.91s/it] 

发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list index out of range
发生异常: list